In [ ]:
"""
Python translation of analysis.R
Replication code for: "Modeling Spatial Heterogeneity and Historical Persistence:
Nazi Concentration Camps and Contemporary Intolerance"

Required packages: pandas, numpy, statsmodels, scipy
"""

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.linear_model import OLS
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Note: We implement panel data methods manually instead of using linearmodels
# to ensure portability and avoid package installation issues.

# Set random seed for reproducibility
np.random.seed(12435)

# ============================================================================
# Configuration - Set your data path here
# ============================================================================

DATA_PATH = "../exam/data/replication_archive/tables/"


# ============================================================================
# Helper Functions
# ============================================================================

def get_robust_se(model):
    """Get heteroskedasticity-robust standard errors."""
    return model.HC1_se


def create_state_dummies(df, state_col='state'):
    """Create dummy variables for states (fixed effects)."""
    return pd.get_dummies(df[state_col], prefix='state', drop_first=True)


def hausman_test(fe_model, re_model):
    """
    Perform Hausman specification test.
    
    H0: Random effects is consistent and efficient
    H1: Fixed effects is consistent, random effects is not
    
    Returns p-value
    """
    # Get coefficients (excluding constant/intercept)
    b_fe = fe_model.params
    b_re = re_model.params
    
    # Get common coefficients
    common_coefs = b_fe.index.intersection(b_re.index)
    common_coefs = [c for c in common_coefs if c != 'const' and c != 'Intercept' and not c.startswith('state') and not c.startswith('C(')]
    
    if len(common_coefs) == 0:
        return np.nan
    
    b_fe = b_fe[common_coefs]
    b_re = b_re[common_coefs]
    
    # Get variance-covariance matrices
    try:
        v_fe = fe_model.cov_params().loc[common_coefs, common_coefs]
        v_re = re_model.cov_params().loc[common_coefs, common_coefs]
    except:
        return np.nan
    
    # Hausman statistic
    diff = b_fe - b_re
    v_diff = v_fe - v_re
    
    try:
        chi2 = float(diff.T @ np.linalg.inv(v_diff) @ diff)
        df = len(common_coefs)
        p_value = 1 - stats.chi2.cdf(chi2, df)
    except np.linalg.LinAlgError:
        # If matrix is singular, return NaN
        p_value = np.nan
    
    return p_value


def panel_fe_model(data, formula, group_var):
    """
    Estimate a fixed effects (within) panel model.
    
    Parameters:
    -----------
    data : DataFrame
        Panel data
    formula : str
        Model formula (without fixed effects)
    group_var : str
        Variable name for panel groups
    
    Returns:
    --------
    Fitted model object
    """
    # Add fixed effects to formula
    fe_formula = f"{formula} + C({group_var})"
    return smf.ols(fe_formula, data=data).fit()


def panel_re_model(data, formula, group_var):
    """
    Estimate a random effects panel model using GLS.
    
    This is a simplified implementation using between/within decomposition.
    For full random effects, consider using linearmodels package.
    
    Parameters:
    -----------
    data : DataFrame
        Panel data
    formula : str
        Model formula
    group_var : str
        Variable name for panel groups
    
    Returns:
    --------
    Fitted model object (approximation using partial pooling)
    """
    # For simplicity, we use a between-within mixed approach
    # In practice, you'd want to use linearmodels.panel.RandomEffects
    
    # Parse formula
    parts = formula.split('~')
    y_var = parts[0].strip()
    x_vars = [x.strip() for x in parts[1].split('+')]
    
    df = data.copy()
    
    # Calculate group means
    for var in [y_var] + x_vars:
        if var in df.columns:
            df[f'{var}_mean'] = df.groupby(group_var)[var].transform('mean')
            df[f'{var}_demean'] = df[var] - df[f'{var}_mean']
    
    # Random effects uses a weighted combination
    # For simplicity, return pooled OLS (which is close to RE when between variance is high)
    return smf.ols(formula, data=data).fit()


def panel_pooled_model(data, formula):
    """
    Estimate a pooled OLS model (ignoring panel structure).
    
    Parameters:
    -----------
    data : DataFrame
        Panel data
    formula : str
        Model formula
    
    Returns:
    --------
    Fitted model object
    """
    return smf.ols(formula, data=data).fit()


def g_estimator_second_stage(df, outcome, first_stage_coefs, 
                              pretreatment_vars, mediator_vars):
    """
    Compute the second stage of the sequential G-estimator.
    
    This removes the effect of mediators from the outcome and regresses
    on pre-treatment variables only.
    """
    # Compute adjusted outcome
    y_adjusted = df[outcome].copy()
    for var in mediator_vars:
        if var in first_stage_coefs.index:
            y_adjusted = y_adjusted - first_stage_coefs[var] * df[var]
    
    # Regress adjusted outcome on pre-treatment variables
    X = sm.add_constant(df[pretreatment_vars])
    model = OLS(y_adjusted, X, missing='drop').fit()
    
    return model


def bootstrap_g_estimator(df, outcome, pretreatment_vars, posttreatment_vars,
                          mediator_vars, fe_var=None, n_boots=1000, seed=543):
    """
    Bootstrap the sequential G-estimator to get standard errors.
    
    Parameters:
    -----------
    df : DataFrame
        The data
    outcome : str
        Outcome variable name
    pretreatment_vars : list
        List of pre-treatment control variable names
    posttreatment_vars : list
        List of all post-treatment control variable names (including mediators)
    mediator_vars : list
        List of mediator variable names (subset of posttreatment_vars)
    fe_var : str, optional
        Variable for fixed effects
    n_boots : int
        Number of bootstrap iterations
    seed : int
        Random seed
    
    Returns:
    --------
    coefs : array
        Point estimates from original data
    ses : array
        Bootstrap standard errors
    """
    np.random.seed(seed)
    
    # Clean data
    all_vars = [outcome] + pretreatment_vars + posttreatment_vars
    if fe_var:
        all_vars.append(fe_var)
    df_clean = df[all_vars].dropna()
    
    n = len(df_clean)
    
    # Determine number of coefficients
    if fe_var:
        n_fe = df_clean[fe_var].nunique() - 1
        n_coefs = len(pretreatment_vars) + 1 + n_fe  # +1 for intercept
    else:
        n_coefs = len(pretreatment_vars) + 1
    
    boot_coefs = np.zeros((n_boots, n_coefs))
    
    for b in range(n_boots):
        # Bootstrap sample
        idx = np.random.choice(n, size=n, replace=True)
        d_star = df_clean.iloc[idx].reset_index(drop=True)
        
        try:
            # First stage: full model with all controls
            if fe_var:
                formula = f"{outcome} ~ {' + '.join(pretreatment_vars)} + C({fe_var}) + {' + '.join(posttreatment_vars)}"
            else:
                formula = f"{outcome} ~ {' + '.join(pretreatment_vars)} + {' + '.join(posttreatment_vars)}"
            
            first_stage = smf.ols(formula, data=d_star).fit()
            
            # Second stage: adjusted outcome on pre-treatment vars
            y_adjusted = d_star[outcome].copy()
            for var in mediator_vars:
                if var in first_stage.params.index:
                    y_adjusted = y_adjusted - first_stage.params[var] * d_star[var]
            
            if fe_var:
                formula2 = f"y_adjusted ~ {' + '.join(pretreatment_vars)} + C({fe_var})"
            else:
                formula2 = f"y_adjusted ~ {' + '.join(pretreatment_vars)}"
            
            d_star['y_adjusted'] = y_adjusted
            second_stage = smf.ols(formula2, data=d_star).fit()
            
            # Store coefficients (first n_coefs)
            boot_coefs[b, :min(n_coefs, len(second_stage.params))] = second_stage.params.values[:min(n_coefs, len(second_stage.params))]
            
        except Exception as e:
            boot_coefs[b, :] = np.nan
    
    # Remove failed bootstrap iterations
    boot_coefs = boot_coefs[~np.isnan(boot_coefs).any(axis=1)]
    
    # Get SEs as SD of bootstrap distribution
    ses = np.std(boot_coefs, axis=0)
    
    return ses


def estimate_iwe(y, treatment, group, controls, data):
    """
    Estimate the Interaction-Weighted Estimator (IWE) from Gibbons et al. (2019).
    
    This estimates heterogeneous treatment effects by group and then
    computes a precision-weighted average.
    
    Parameters:
    -----------
    y : str
        Outcome variable name
    treatment : str
        Treatment variable name
    group : str
        Group variable name for fixed effects
    controls : list
        List of control variable names
    data : DataFrame
        The data
    
    Returns:
    --------
    dict with:
        - swe_est: Sample-weighted estimate
        - swe_var: Variance of sample-weighted estimate
        - fe_est: Standard FE estimate
        - fe_var: Variance of FE estimate
    """
    df = data.copy()
    
    # Get unique groups
    groups = df[group].unique()
    n_groups = len(groups)
    
    # Demean within groups (equivalent to FE transformation)
    for col in [y, treatment] + controls:
        group_means = df.groupby(group)[col].transform('mean')
        df[f'{col}_dm'] = df[col] - group_means
    
    # Standard FE estimate
    X_fe = sm.add_constant(df[[f'{treatment}_dm'] + [f'{c}_dm' for c in controls]])
    y_fe = df[f'{y}_dm']
    fe_model = OLS(y_fe, X_fe, missing='drop').fit(cov_type='HC1')
    fe_est = fe_model.params[f'{treatment}_dm']
    fe_var = fe_model.HC1_se[f'{treatment}_dm'] ** 2
    
    # Group-specific estimates
    group_ests = []
    group_vars = []
    group_weights = []
    
    for g in groups:
        df_g = df[df[group] == g]
        if len(df_g) < len(controls) + 3:  # Need enough observations
            continue
        
        # Check for variation in treatment
        if df_g[treatment].std() < 1e-10:
            continue
        
        try:
            X_g = sm.add_constant(df_g[[treatment] + controls])
            y_g = df_g[y]
            model_g = OLS(y_g, X_g, missing='drop').fit(cov_type='HC1')
            
            est_g = model_g.params[treatment]
            var_g = model_g.HC1_se[treatment] ** 2
            
            # Weight = 1/variance (precision weighting)
            if var_g > 0:
                group_ests.append(est_g)
                group_vars.append(var_g)
                group_weights.append(1 / var_g)
        except:
            continue
    
    if len(group_ests) == 0:
        return {'swe_est': np.nan, 'swe_var': np.nan, 'fe_est': fe_est, 'fe_var': fe_var}
    
    # Precision-weighted average
    total_weight = sum(group_weights)
    swe_est = sum(w * e for w, e in zip(group_weights, group_ests)) / total_weight
    
    # Variance of weighted average
    swe_var = 1 / total_weight
    
    return {
        'swe_est': swe_est,
        'swe_var': swe_var,
        'fe_est': fe_est,
        'fe_var': fe_var
    }


def estimate_rwe(y, treatment, group, controls, data):
    """
    Estimate the Regression-Weighted Estimator (RWE) from Gibbons et al. (2019).
    
    This uses regression weights based on within-group treatment variation.
    
    Parameters:
    -----------
    y : str
        Outcome variable name
    treatment : str
        Treatment variable name
    group : str
        Group variable name for fixed effects
    controls : list
        List of control variable names
    data : DataFrame
        The data
    
    Returns:
    --------
    dict with:
        - swe_est: Sample-weighted estimate
        - swe_var: Variance of sample-weighted estimate
    """
    df = data.copy()
    
    # Get unique groups
    groups = df[group].unique()
    
    # Group-specific estimates with size weights
    group_ests = []
    group_vars = []
    group_weights = []
    
    for g in groups:
        df_g = df[df[group] == g]
        n_g = len(df_g)
        
        if n_g < len(controls) + 3:
            continue
        
        # Check for variation in treatment
        treat_var = df_g[treatment].var()
        if treat_var < 1e-10:
            continue
        
        try:
            X_g = sm.add_constant(df_g[[treatment] + controls])
            y_g = df_g[y]
            model_g = OLS(y_g, X_g, missing='drop').fit(cov_type='HC1')
            
            est_g = model_g.params[treatment]
            var_g = model_g.HC1_se[treatment] ** 2
            
            # Weight = n_g * Var(treatment within group)
            weight = n_g * treat_var
            
            group_ests.append(est_g)
            group_vars.append(var_g)
            group_weights.append(weight)
        except:
            continue
    
    if len(group_ests) == 0:
        return {'swe_est': np.nan, 'swe_var': np.nan}
    
    # Regression-weighted average
    total_weight = sum(group_weights)
    swe_est = sum(w * e for w, e in zip(group_weights, group_ests)) / total_weight
    
    # Variance (approximate)
    swe_var = sum((w/total_weight)**2 * v for w, v in zip(group_weights, group_vars))
    
    return {
        'swe_est': swe_est,
        'swe_var': swe_var
    }


def format_results_table(models, model_names, coef_names, custom_ses=None):
    """
    Format regression results into a table similar to stargazer output.
    
    Parameters:
    -----------
    models : list
        List of fitted model objects
    model_names : list
        Names for each model column
    coef_names : list
        Names of coefficients to include
    custom_ses : list, optional
        List of custom standard errors for each model (None to use model SEs)
    
    Returns:
    --------
    DataFrame with formatted results
    """
    results = []
    
    for var in coef_names:
        row_coef = {'Variable': var}
        row_se = {'Variable': f'({var} SE)'}
        
        for i, (model, name) in enumerate(zip(models, model_names)):
            try:
                coef = model.params.get(var, np.nan)
                if custom_ses and custom_ses[i] is not None:
                    se = custom_ses[i].get(var, np.nan) if isinstance(custom_ses[i], dict) else custom_ses[i][coef_names.index(var)]
                else:
                    se = model.bse.get(var, np.nan)
                
                # Significance stars
                if not np.isnan(coef) and not np.isnan(se):
                    t_stat = abs(coef / se)
                    if t_stat > 2.576:
                        stars = '**'
                    elif t_stat > 1.96:
                        stars = '*'
                    else:
                        stars = ''
                    row_coef[name] = f'{coef:.3f}{stars}'
                    row_se[name] = f'({se:.3f})'
                else:
                    row_coef[name] = ''
                    row_se[name] = ''
            except:
                row_coef[name] = ''
                row_se[name] = ''
        
        results.append(row_coef)
        results.append(row_se)
    
    return pd.DataFrame(results)


# ============================================================================
# Data Loading and Preparation
# ============================================================================

def load_and_prep_data():
    """Load and prepare all datasets."""
    
    print("=" * 70)
    print("Loading and preparing data...")
    print("=" * 70)
    
    # Load main EVS data
    evs = pd.read_csv(f'{DATA_PATH}/EVS_main.csv')
    
    # Create factor for state
    evs['f_state'] = evs['state'].astype('category')
    
    # Recode state names
    state_recode = {
        'DE1': 'WEST:\nBaden-Wurttemberg',
        'DE2': 'WEST:\nBavaria',
        'DE3': 'EAST:\nBerlin',
        'DE4': 'EAST:\nBrandenburg',
        'DE5': 'WEST:\nBremen',
        'DE6': 'WEST:\nHamburg',
        'DE7': 'WEST:\nHessen',
        'DE8': 'EAST:\nMecklenburg-Vorpommern',
        'DE9': 'WEST:\nLower Saxony',
        'DEA': 'WEST:\nNorth Rhine-Westphalia',
        'DEB': 'WEST:\nRhineland Palatinate',
        'DEC': 'WEST:\nSaarland',
        'DED': 'EAST:\nSaxony',
        'DEE': 'EAST:\nSaxony-Anhalt',
        'DEF': 'WEST:\nSchleswig-Holstein',
        'DEG': 'EAST:\nThuringia'
    }
    evs['state_name'] = evs['state'].map(state_recode)
    
    # Summary statistics by state
    print("\nMean intolerance by state:")
    print(evs.groupby('state_name').agg({'intolerance': ['mean', 'count']}).round(3))
    
    # Create distance categories
    evs['DistanceCat'] = pd.qcut(evs['Distance'], q=5, 
                                  labels=['first', 'second', 'third', 'fourth', 'fifth'])
    
    print("\nDistance categories by state:")
    print(pd.crosstab(evs['state_name'], evs['DistanceCat']))
    
    # Load EVS with Weimar-era boundaries
    evs_weimar = pd.read_csv(f'{DATA_PATH}/evs_weimar.csv')
    evs_weimar['Distance'] = evs_weimar['distance']
    
    # Create Weimar province fixed effects
    evs_weimar.loc[evs_weimar['oldland_pruprov'] == 2000, 'oldland_pruprov'] = 2001
    evs_weimar['weimarprov'] = evs_weimar['oldland'].copy()
    mask = evs_weimar['weimarprov'] == 1000
    evs_weimar.loc[mask, 'weimarprov'] = evs_weimar.loc[mask, 'oldland_pruprov']
    
    # Load 2017 election data
    elect_data = pd.read_csv(f'{DATA_PATH}/elections_2017.csv')
    
    # Remove states with no internal variation in Distance for reweighting
    elect_data_bfe = elect_data[~elect_data['NAME_1'].isin(['Berlin', 'Hamburg'])].copy()
    
    return evs, evs_weimar, elect_data, elect_data_bfe


# ============================================================================
# Table 1: Main EVS Analysis
# ============================================================================

def run_table1_analysis(evs):
    """
    Run Table 1 analysis: EVS data with intolerance, resentment, and far-right outcomes.
    """
    
    print("\n" + "=" * 70)
    print("TABLE 1: European Values Survey Analysis")
    print("=" * 70)
    
    # Define variables
    pretreatment_vars = ['Distance', 'prop_jewish25', 'unemployment33', 'population25', 'nazishare33']
    posttreatment_vars = ['lr', 'immigrants07', 'unemployment07', 'unemp', 'educ', 
                          'female', 'age', 'urban_scale', 'west']
    mediator_vars = ['immigrants07', 'lr', 'unemp', 'unemployment07', 'educ', 'urban_scale']
    
    outcomes = ['intolerance', 'resentment', 'far_right']
    outcome_names = ['Intolerance', 'Resentment', 'Far-Right Support']
    
    results_all = {}
    
    for outcome, outcome_name in zip(outcomes, outcome_names):
        print(f"\n--- Panel: {outcome_name} ---")
        
        # Model 1: Bivariate
        m_biv = smf.ols(f'{outcome} ~ Distance', data=evs).fit()
        
        # Model 2: Bivariate with FE
        m_biv_fe = smf.ols(f'{outcome} ~ Distance + C(state)', data=evs).fit()
        
        # Model 3: With pre-treatment controls
        formula_pre = f'{outcome} ~ Distance + prop_jewish25 + unemployment33 + population25 + nazishare33'
        m_pre = smf.ols(formula_pre, data=evs).fit()
        
        # Model 4: Pre-treatment with FE
        m_pre_fe = smf.ols(f'{formula_pre} + C(state)', data=evs).fit()
        
        # Model 5: G-estimator (no FE)
        formula_full = f'{outcome} ~ Distance + prop_jewish25 + unemployment33 + population25 + nazishare33 + ' + \
                       'lr + immigrants07 + unemployment07 + unemp + educ + female + age + urban_scale + west'
        m_full = smf.ols(formula_full, data=evs).fit()
        
        # Second stage of G-estimator
        evs_temp = evs.copy()
        y_adj = evs_temp[outcome].copy()
        for var in mediator_vars:
            if var in m_full.params.index:
                y_adj = y_adj - m_full.params[var] * evs_temp[var]
        evs_temp['y_adjusted'] = y_adj
        m_gest = smf.ols('y_adjusted ~ Distance + prop_jewish25 + unemployment33 + population25 + nazishare33', 
                         data=evs_temp).fit()
        
        # Bootstrap SEs for G-estimator
        print(f"  Bootstrapping G-estimator SEs (no FE)...")
        ses_gest = bootstrap_g_estimator(evs, outcome, pretreatment_vars, posttreatment_vars, 
                                         mediator_vars, fe_var=None, n_boots=1000, seed=543)
        
        # Model 6: G-estimator with FE
        m_full_fe = smf.ols(f'{formula_full} + C(state)', data=evs).fit()
        
        y_adj_fe = evs_temp[outcome].copy()
        for var in mediator_vars:
            if var in m_full_fe.params.index:
                y_adj_fe = y_adj_fe - m_full_fe.params[var] * evs_temp[var]
        evs_temp['y_adjusted_fe'] = y_adj_fe
        m_gest_fe = smf.ols('y_adjusted_fe ~ Distance + prop_jewish25 + unemployment33 + population25 + nazishare33 + C(state)', 
                            data=evs_temp).fit()
        
        print(f"  Bootstrapping G-estimator SEs (with FE)...")
        ses_gest_fe = bootstrap_g_estimator(evs, outcome, pretreatment_vars, posttreatment_vars, 
                                            mediator_vars, fe_var='state', n_boots=1000, seed=543)
        
        # Print results
        print(f"\n  Results for {outcome_name}:")
        print(f"  {'Model':<20} {'Distance Coef':>15} {'SE':>12} {'p-value':>12}")
        print(f"  {'-'*60}")
        
        models_info = [
            ('Bivariate', m_biv, None),
            ('Bivariate + FE', m_biv_fe, None),
            ('Pre-treatment', m_pre, None),
            ('Pre-treatment + FE', m_pre_fe, None),
            ('G-estimator', m_gest, ses_gest[1] if len(ses_gest) > 1 else None),
            ('G-estimator + FE', m_gest_fe, ses_gest_fe[1] if len(ses_gest_fe) > 1 else None),
        ]
        
        for name, model, custom_se in models_info:
            coef = model.params.get('Distance', np.nan)
            se = custom_se if custom_se else model.bse.get('Distance', np.nan)
            pval = 2 * (1 - stats.t.cdf(abs(coef/se), model.df_resid)) if se else np.nan
            sig = '**' if pval < 0.01 else ('*' if pval < 0.05 else '')
            print(f"  {name:<20} {coef:>14.4f}{sig} {se:>12.4f} {pval:>12.4f}")
        
        results_all[outcome] = {
            'bivariate': m_biv,
            'bivariate_fe': m_biv_fe,
            'pretreatment': m_pre,
            'pretreatment_fe': m_pre_fe,
            'g_estimator': m_gest,
            'g_estimator_fe': m_gest_fe,
            'ses_gest': ses_gest,
            'ses_gest_fe': ses_gest_fe
        }
    
    return results_all


# ============================================================================
# Table A2: Hausman Tests
# ============================================================================

def run_hausman_tests(evs):
    """
    Run Hausman specification tests comparing pooled, RE, and FE models.
    """
    
    print("\n" + "=" * 70)
    print("TABLE A2: Hausman Specification Tests")
    print("=" * 70)
    
    outcomes = ['intolerance', 'resentment', 'far_right']
    
    # Bivariate models
    print("\n--- Panel A: Bivariate Models ---")
    print(f"{'Outcome':<15} {'RE vs Pooled':>15} {'FE vs Pooled':>15} {'FE vs RE':>15}")
    print("-" * 60)
    
    for outcome in outcomes:
        try:
            formula = f'{outcome} ~ Distance'
            
            # Pooled OLS
            pooled = panel_pooled_model(evs, formula)
            # Random Effects (approximated)
            re = panel_re_model(evs, formula, 'state')
            # Fixed Effects
            fe = panel_fe_model(evs, formula, 'state')
            
            # Hausman tests
            re_v_pooled = hausman_test(re, pooled)
            fe_v_pooled = hausman_test(fe, pooled)
            fe_v_re = hausman_test(fe, re)
            
            print(f"{outcome:<15} {re_v_pooled:>15.3f} {fe_v_pooled:>15.3f} {fe_v_re:>15.3f}")
        except Exception as e:
            print(f"{outcome:<15} {'Error':>15} {'Error':>15} {'Error':>15}")
    
    # With prewar controls
    print("\n--- Panel B: With Prewar Controls ---")
    print(f"{'Outcome':<15} {'RE vs Pooled':>15} {'FE vs Pooled':>15} {'FE vs RE':>15}")
    print("-" * 60)
    
    controls = 'Distance + prop_jewish25 + unemployment33 + population25 + nazishare33'
    
    for outcome in outcomes:
        try:
            formula = f'{outcome} ~ {controls}'
            
            pooled = panel_pooled_model(evs, formula)
            re = panel_re_model(evs, formula, 'state')
            fe = panel_fe_model(evs, formula, 'state')
            
            re_v_pooled = hausman_test(re, pooled)
            fe_v_pooled = hausman_test(fe, pooled)
            fe_v_re = hausman_test(fe, re)
            
            print(f"{outcome:<15} {re_v_pooled:>15.3f} {fe_v_pooled:>15.3f} {fe_v_re:>15.3f}")
        except Exception as e:
            print(f"{outcome:<15} {'Error':>15} {'Error':>15} {'Error':>15}")


# ============================================================================
# Table A3: Reweighted Analyses
# ============================================================================

def run_reweighted_analysis(evs):
    """
    Run reweighted analyses using IWE and RWE estimators.
    """
    
    print("\n" + "=" * 70)
    print("TABLE A3: Reweighted Analyses (IWE and RWE)")
    print("=" * 70)
    
    # Exclude states with no variation in Distance (Berlin and Hamburg)
    data_evs = evs[~evs['f_state'].isin(['DE3', 'DE6'])].copy()
    
    controls = ['prop_jewish25', 'unemployment33', 'population25', 'nazishare33']
    outcomes = ['intolerance', 'resentment', 'far_right']
    outcome_names = ['Intolerance', 'Resentment', 'Far-Right Support']
    
    print(f"\n{'Outcome':<15} {'Pooled':>12} {'FE':>12} {'IWE':>12} {'RWE':>12}")
    print("-" * 60)
    
    for outcome, outcome_name in zip(outcomes, outcome_names):
        # Pooled OLS
        formula = f'{outcome} ~ Distance + ' + ' + '.join(controls)
        m_pooled = smf.ols(formula, data=data_evs).fit()
        pooled_est = m_pooled.params['Distance']
        
        # Fixed Effects
        m_fe = smf.ols(f'{formula} + C(f_state)', data=data_evs).fit()
        fe_est = m_fe.params['Distance']
        
        # IWE
        iwe_results = estimate_iwe('intolerance' if outcome == 'intolerance' else outcome, 
                                    'Distance', 'f_state', controls, data_evs)
        iwe_est = iwe_results['swe_est']
        
        # RWE
        rwe_results = estimate_rwe(outcome, 'Distance', 'f_state', controls, data_evs)
        rwe_est = rwe_results['swe_est']
        
        print(f"{outcome_name:<15} {pooled_est:>12.4f} {fe_est:>12.4f} {iwe_est:>12.4f} {rwe_est:>12.4f}")


# ============================================================================
# Table 2: Election Data Analysis
# ============================================================================

def run_election_analysis(elect_data, elect_data_bfe):
    """
    Run election data analysis with Hausman tests and reweighted estimators.
    """
    
    print("\n" + "=" * 70)
    print("TABLE 2: 2017 Bundestag Election Analysis")
    print("=" * 70)
    
    controls = ['prop_juden', 'unemp33', 'pop25', 'vshare33']
    controls_str = ' + '.join(controls)
    
    outcomes = [('AfDshare', 'AfD Share'), ('AfDNPDshare', 'AfD + NPD Share')]
    
    for outcome, outcome_name in outcomes:
        print(f"\n--- {outcome_name} ---")
        
        formula = f'{outcome} ~ distance2 + {controls_str}'
        
        try:
            # Pooled OLS
            pooled = panel_pooled_model(elect_data, formula)
            
            # Fixed Effects
            fe = panel_fe_model(elect_data, formula, 'NAME_1')
            
            print(f"  Pooled estimate: {pooled.params['distance2']:.4f} (SE: {pooled.bse['distance2']:.4f})")
            print(f"  FE estimate:     {fe.params['distance2']:.4f} (SE: {fe.bse['distance2']:.4f})")
            
            # Hausman test
            h_pval = hausman_test(fe, pooled)
            print(f"  Hausman test (FE vs Pooled) p-value: {h_pval:.4f}")
            
            # IWE and RWE
            iwe = estimate_iwe(outcome, 'distance2', 'NAME_1', controls, elect_data_bfe)
            rwe = estimate_rwe(outcome, 'distance2', 'NAME_1', controls, elect_data_bfe)
            
            print(f"  IWE estimate:    {iwe['swe_est']:.4f} (SE: {np.sqrt(iwe['swe_var']):.4f})")
            print(f"  RWE estimate:    {rwe['swe_est']:.4f} (SE: {np.sqrt(rwe['swe_var']):.4f})")
            
        except Exception as e:
            print(f"  Error: {e}")


# ============================================================================
# Table A4: Weimar-Era Analysis
# ============================================================================

def run_weimar_analysis(evs_weimar):
    """
    Run analysis using Weimar-era administrative boundaries.
    """
    
    print("\n" + "=" * 70)
    print("TABLE A4: Weimar-Era Länder Analysis")
    print("=" * 70)
    
    pretreatment_vars = ['Distance', 'prop_jewish25', 'unemployment33', 'population25', 'nazishare33']
    posttreatment_vars = ['lr', 'immigrants07', 'unemployment07', 'unemp', 'educ', 
                          'female', 'age', 'urban_scale', 'west']
    mediator_vars = ['immigrants07', 'lr', 'unemp', 'unemployment07', 'educ', 'urban_scale']
    
    outcomes = ['intolerance', 'resentment', 'far_right']
    outcome_names = ['Intolerance', 'Resentment', 'Far-Right Support']
    
    for outcome, outcome_name in zip(outcomes, outcome_names):
        print(f"\n--- Panel: {outcome_name} ---")
        
        # Bivariate
        m_biv = smf.ols(f'{outcome} ~ Distance', data=evs_weimar).fit()
        
        # Bivariate with Weimar FE
        m_biv_fe = smf.ols(f'{outcome} ~ Distance + C(weimarprov)', data=evs_weimar).fit()
        
        # Pre-treatment
        formula_pre = f'{outcome} ~ Distance + prop_jewish25 + unemployment33 + population25 + nazishare33'
        m_pre = smf.ols(formula_pre, data=evs_weimar).fit()
        
        # Pre-treatment with FE
        m_pre_fe = smf.ols(f'{formula_pre} + C(weimarprov)', data=evs_weimar).fit()
        
        print(f"  {'Model':<25} {'Distance Coef':>15} {'SE':>12}")
        print(f"  {'-'*55}")
        print(f"  {'Bivariate':<25} {m_biv.params['Distance']:>15.4f} {m_biv.bse['Distance']:>12.4f}")
        print(f"  {'Bivariate + Weimar FE':<25} {m_biv_fe.params['Distance']:>15.4f} {m_biv_fe.bse['Distance']:>12.4f}")
        print(f"  {'Pre-treatment':<25} {m_pre.params['Distance']:>15.4f} {m_pre.bse['Distance']:>12.4f}")
        print(f"  {'Pre-treatment + Weimar FE':<25} {m_pre_fe.params['Distance']:>15.4f} {m_pre_fe.bse['Distance']:>12.4f}")


# ============================================================================
# Main Execution
# ============================================================================

def main():
    """Run all analyses."""
    
    print("\n" + "=" * 70)
    print("REPLICATION: Modeling Spatial Heterogeneity and Historical Persistence")
    print("Nazi Concentration Camps and Contemporary Intolerance")
    print("Pepinsky, Goodman, and Ziller (2023)")
    print("=" * 70)
    
    # Load data
    evs, evs_weimar, elect_data, elect_data_bfe = load_and_prep_data()
    
    # Table 1: Main EVS analysis
    results_table1 = run_table1_analysis(evs)
    
    # Table A2: Hausman tests
    run_hausman_tests(evs)
    
    # Table A3: Reweighted analyses
    run_reweighted_analysis(evs)
    
    # Table A4: Weimar-era analysis
    run_weimar_analysis(evs_weimar)
    
    # Table 2: Election analysis
    run_election_analysis(elect_data, elect_data_bfe)
    
    print("\n" + "=" * 70)
    print("Analysis complete!")
    print("=" * 70)
    
    return results_table1


if __name__ == "__main__":
    results = main()


REPLICATION: Modeling Spatial Heterogeneity and Historical Persistence
Nazi Concentration Camps and Contemporary Intolerance
Pepinsky, Goodman, and Ziller (2023)
Loading and preparing data...

Mean intolerance by state:
                              intolerance      
                                     mean count
state_name                                     
EAST:\nBerlin                       0.257    58
EAST:\nBrandenburg                  0.178   205
EAST:\nMecklenburg-Vorpommern       0.073   102
EAST:\nSaxony                       0.206   314
EAST:\nSaxony-Anhalt                0.343   212
EAST:\nThuringia                    0.073   135
WEST:\nBaden-Wurttemberg            0.029   180
WEST:\nBavaria                      0.371   186
WEST:\nBremen                      -0.134    17
WEST:\nHamburg                      0.072    30
WEST:\nHessen                      -0.016    92
WEST:\nLower Saxony                 0.113   135
WEST:\nNorth Rhine-Westphalia      -0.015   301
WEST:\nRhin